# Human-in-the-Loop Arbetsflöde med Microsoft Agent Framework

## 🎯 Inlärningsmål

I denna notebook kommer du att lära dig hur man implementerar **human-in-the-loop** arbetsflöden med hjälp av Microsoft Agent Frameworks `RequestInfoExecutor`. Detta kraftfulla mönster gör det möjligt att pausa AI-arbetsflöden för att samla in mänsklig input, vilket gör dina agenter interaktiva och ger människor kontroll över kritiska beslut.

## 🔄 Vad är Human-in-the-Loop?

**Human-in-the-loop (HITL)** är ett designmönster där AI-agenter pausar utförandet för att begära mänsklig input innan de fortsätter. Detta är viktigt för:

- ✅ **Kritiska beslut** - Få mänskligt godkännande innan viktiga åtgärder vidtas
- ✅ **Otydliga situationer** - Låt människor förtydliga när AI är osäker
- ✅ **Användarpreferenser** - Fråga användare om att välja mellan flera alternativ
- ✅ **Efterlevnad & säkerhet** - Säkerställ mänsklig övervakning för reglerade operationer
- ✅ **Interaktiva upplevelser** - Bygg konversationsagenter som svarar på användarens input

## 🏗️ Hur det fungerar i Microsoft Agent Framework

Ramverket tillhandahåller tre nyckelkomponenter för HITL:

1. **`RequestInfoExecutor`** - En speciell executor som pausar arbetsflödet och skickar ut en `RequestInfoEvent`
2. **`RequestInfoMessage`** - Basklass för typade begäran som skickas till människor
3. **`RequestResponse`** - Korrelerar mänskliga svar med ursprungliga begäran med hjälp av `request_id`

**Arbetsflödesmönster:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Vårt exempel: Hotellbokning med användarbekräftelse

Vi bygger vidare på det villkorliga arbetsflödet genom att lägga till mänsklig bekräftelse **innan** vi föreslår alternativa destinationer:

1. Användaren begär en destination (t.ex. "Paris")
2. `availability_agent` kontrollerar om rum är tillgängliga
3. **Om inga rum finns** → `confirmation_agent` frågar "Vill du se alternativ?"
4. Arbetsflödet **pausas** med hjälp av `RequestInfoExecutor`
5. **Människan svarar** "ja" eller "nej" via konsolinmatning
6. `decision_manager` styr baserat på svaret:
   - **Ja** → Visa alternativa destinationer
   - **Nej** → Avbryt bokningsförfrågan
7. Visa slutresultatet

Detta visar hur man ger användare kontroll över agentens förslag!

---

Låt oss sätta igång! 🚀


## Steg 1: Importera nödvändiga bibliotek

Vi importerar standardkomponenterna för Agent Framework plus **specifika klasser för människa-i-loopen**:
- `RequestInfoExecutor` - Utförare som pausar arbetsflödet för mänsklig input
- `RequestInfoEvent` - Händelse som utlöses när mänsklig input efterfrågas
- `RequestInfoMessage` - Basklass för typade begäran-payloads
- `RequestResponse` - Korrelerar mänskliga svar med begäran
- `WorkflowOutputEvent` - Händelse för att upptäcka arbetsflödesutgångar


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## Steg 2: Definiera Pydantic-modeller för strukturerade utdata

Dessa modeller definierar **schemat** som agenter kommer att returnera. Vi behåller alla modeller från det villkorliga arbetsflödet och lägger till:

**Nytt för Human-in-the-Loop:**
- `HumanFeedbackRequest` - Underklass av `RequestInfoMessage` som definierar begärans nyttolast som skickas till människor
  - Innehåller `prompt` (frågan att ställa) och `destination` (kontext om den otillgängliga staden)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## Steg 3: Skapa verktyget för hotellbokning

Samma verktyg från det villkorliga arbetsflödet - kontrollerar om rum är tillgängliga på destinationen.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Steg 4: Definiera villkorsfunktioner för routing

Vi behöver **fyra villkorsfunktioner** för vårt human-in-the-loop-arbetsflöde:

**Från det villkorliga arbetsflödet:**
1. `has_availability_condition` - Riktar när hotell ÄR tillgängliga
2. `no_availability_condition` - Riktar när hotell INTE är tillgängliga

**Nytt för human-in-the-loop:**
3. `user_wants_alternatives_condition` - Riktar när användaren säger "ja" till alternativ
4. `user_declines_alternatives_condition` - Riktar när användaren säger "nej" till alternativ


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## Steg 5: Skapa Decision Manager Executor

Detta är **kärnan i human-in-the-loop-mönstret**! `DecisionManager` är en anpassad `Executor` som:

1. **Tar emot mänsklig feedback** via `RequestResponse`-objekt
2. **Bearbetar användarens beslut** (ja/nej)
3. **Styr arbetsflödet** genom att skicka meddelanden till lämpliga agenter

Nyckelfunktioner:
- Använder dekoratorn `@handler` för att exponera metoder som arbetsflödessteg
- Tar emot `RequestResponse[HumanFeedbackRequest, str]` som innehåller både den ursprungliga begäran och användarens svar
- Genererar enkla "ja" eller "nej"-meddelanden som aktiverar våra villkorsfunktioner


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## Steg 6: Skapa anpassad visningsutförare

Samma visningsutförare från det villkorliga arbetsflödet - ger slutresultat som arbetsflödesutdata.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## Steg 7: Ladda miljövariabler

Konfigurera LLM-klienten (GitHub Models, Azure OpenAI eller OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## Steg 8: Skapa AI-agenter och exekutorer

Vi skapar **sex arbetsflödeskomponenter**:

**Agenter (inbäddade i AgentExecutor):**
1. **availability_agent** - Kontrollerar hotellens tillgänglighet med hjälp av verktyget
2. **confirmation_agent** - 🆕 Förbereder begäran om mänsklig bekräftelse
3. **alternative_agent** - Föreslår alternativa städer (när användaren säger ja)
4. **booking_agent** - Uppmuntrar bokning (när rum är tillgängliga)
5. **cancellation_agent** - 🆕 Hanterar avbokningsmeddelande (när användaren säger nej)

**Specialexekutorer:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor` som pausar arbetsflödet för mänsklig input
7. **decision_manager** - 🆕 Anpassad exekutor som styr baserat på mänskligt svar (redan definierad ovan)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## Steg 9: Bygg arbetsflödet med människa-i-slingan

Nu konstruerar vi arbetsflödets graf med **villkorlig styrning** inklusive vägen med människa-i-slingan:

**Arbetsflödesstruktur:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Viktiga kopplingar:**
- `availability_agent → confirmation_agent` (när inga rum finns)
- `confirmation_agent → prepare_human_request` (omvandlingstyp)
- `prepare_human_request → request_info_executor` (paus för människa)
- `request_info_executor → decision_manager` (alltid - tillhandahåller RequestResponse)
- `decision_manager → alternative_agent` (när användaren säger "ja")
- `decision_manager → cancellation_agent` (när användaren säger "nej")
- `availability_agent → booking_agent` (när rum finns tillgängliga)
- Alla vägar slutar vid `display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## Steg 10: Kör Testfall 1 - Stad UTAN Tillgänglighet (Paris med Mänsklig Bekräftelse)

Det här testet demonstrerar **hela cykeln med mänsklig inblandning**:

1. Begär hotell i Paris
2. availability_agent kontrollerar → Inga rum
3. confirmation_agent skapar en fråga riktad till människa
4. request_info_executor **pausar arbetsflödet** och skickar `RequestInfoEvent`
5. **Applikationen upptäcker händelsen och uppmanar användaren i konsolen**
6. Användaren skriver "ja" eller "nej"
7. Applikationen skickar svar via `send_responses_streaming()`
8. decision_manager styr baserat på svaret
9. Slutresultatet visas

**Viktigt Mönster:**
- Använd `workflow.run_stream()` för första iterationen
- Använd `workflow.send_responses_streaming(pending_responses)` för efterföljande iterationer
- Lyssna efter `RequestInfoEvent` för att upptäcka när mänsklig input behövs
- Lyssna efter `WorkflowOutputEvent` för att fånga slutresultat


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## Steg 11: Kör Testfall 2 - Stad MED Tillgänglighet (Stockholm - Ingen Mänsklig Interaktion Krävs)

Detta test visar den **direkta vägen** när rum är tillgängliga:

1. Begär hotell i Stockholm
2. availability_agent kontrollerar → Rum tillgängliga ✅
3. booking_agent föreslår bokning
4. display_result visar bekräftelse
5. **Ingen mänsklig interaktion krävs!**

Arbetsflödet kringgår helt den mänskliga-in-the-loop-vägen när rum är tillgängliga.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Viktiga insikter och bästa praxis för människa-i-loopen

### ✅ Vad du har lärt dig:

#### 1. **RequestInfoExecutor-mönster**
Mönstret för människa-i-loopen i Microsoft Agent Framework använder tre nyckelkomponenter:
- `RequestInfoExecutor` - Pausar arbetsflödet och skickar händelser
- `RequestInfoMessage` - Basklass för typade begäran (subklassa denna!)
- `RequestResponse` - Korrelerar mänskliga svar med ursprungliga begäran

**Viktig förståelse:**
- `RequestInfoExecutor` samlar INTE in input själv - den pausar bara arbetsflödet
- Din applikationskod måste lyssna på `RequestInfoEvent` och samla in input
- Du måste anropa `send_responses_streaming()` med en dict som mappar `request_id` till användarens svar

#### 2. **Mönster för strömmande exekvering**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Händelsedriven arkitektur**
Lyssna på specifika händelser för att kontrollera arbetsflödet:
- `RequestInfoEvent` - Mänsklig input behövs (arbetsflödet pausat)
- `WorkflowOutputEvent` - Slutresultat är tillgängligt (arbetsflödet klart)
- `WorkflowStatusEvent` - Statusändringar (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS, etc.)

#### 4. **Anpassade exekverare med @handler**
`DecisionManager` visar hur man skapar exekverare som:
- Använder `@handler`-dekoration för att exponera metoder som arbetsflödessteg
- Tar emot typade meddelanden (t.ex. `RequestResponse[HumanFeedbackRequest, str]`)
- Styr arbetsflödet genom att skicka meddelanden till andra exekverare
- Får åtkomst till kontext via `WorkflowContext`

#### 5. **Villkorlig styrning med mänskliga beslut**
Du kan skapa funktionsvillkor som utvärderar mänskliga svar:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Verkliga tillämpningar:

1. **Godkännandeflöden**
   - Få chefens godkännande innan utgiftsrapporter behandlas
   - Kräva mänsklig granskning innan automatiserade e-postmeddelanden skickas
   - Bekräfta transaktioner med högt värde innan exekvering

2. **Innehållsmoderering**
   - Flagga tveksamt innehåll för mänsklig granskning
   - Be moderatorer att fatta slutgiltiga beslut i gränsfall
   - Eskalera till människor när AI:s säkerhet är låg

3. **Kundservice**
   - Låt AI hantera rutinfrågor automatiskt
   - Eskalera komplexa problem till mänskliga agenter
   - Fråga kunden om de vill prata med en människa

4. **Databehandling**
   - Be människor lösa tvetydiga datauppgifter
   - Bekräfta AI:s tolkningar av oklara dokument
   - Låt användare välja mellan flera giltiga tolkningar

5. **Säkerhetskritiska system**
   - Kräva mänsklig bekräftelse innan oåterkalleliga åtgärder
   - Få godkännande innan känsliga data nås
   - Bekräfta beslut i reglerade branscher (sjukvård, finans)

6. **Interaktiva agenter**
   - Bygg konversationsbotar som ställer följdfrågor
   - Skapa guider som leder användare genom komplexa processer
   - Designa agenter som samarbetar med människor steg för steg

### 🔄 Jämförelse: Med vs Utan människa-i-loopen

| Funktion | Villkorligt arbetsflöde | Arbetsflöde med människa-i-loopen |
|----------|--------------------------|-----------------------------------|
| **Exekvering** | Enkel `workflow.run()` | Loop med `run_stream()` + `send_responses_streaming()` |
| **Användarinmatning** | Ingen (helt automatiserad) | Interaktiva frågor via `input()` eller UI |
| **Komponenter** | Agenter + exekverare | + RequestInfoExecutor + DecisionManager |
| **Händelser** | Endast AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent, etc. |
| **Pausning** | Ingen pausning | Arbetsflödet pausas vid RequestInfoExecutor |
| **Mänsklig kontroll** | Ingen mänsklig kontroll | Människor fattar viktiga beslut |
| **Användningsfall** | Automatisering | Samarbete & övervakning |

### 🚀 Avancerade mönster:

#### Flera mänskliga beslutspunkter
Du kan ha flera `RequestInfoExecutor`-noder i samma arbetsflöde:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Timeout-hantering
Implementera tidsgränser för mänskliga svar:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Integration med avancerade UI
Istället för `input()`, integrera med webbaserade UI, Slack, Teams, etc.:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Villkorlig människa-i-loopen
Be om mänsklig input endast i specifika situationer:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ Bästa praxis:

1. **Subclassa alltid RequestInfoMessage**
   - Ger typkontroll och validering
   - Möjliggör rik kontext för UI-rendering
   - Klargör syftet med varje begäran

2. **Använd beskrivande frågor**
   - Inkludera kontext om vad du frågar
   - Förklara konsekvenserna av varje val
   - Håll frågorna enkla och tydliga

3. **Hantera oväntad input**
   - Validera användarsvar
   - Tillhandahåll standardvärden för ogiltig input
   - Ge tydliga felmeddelanden

4. **Spåra begäran-ID**
   - Använd korrelationen mellan request_id och svar
   - Försök inte hantera tillstånd manuellt

5. **Designa för icke-blockering**
   - Blockera inte trådar som väntar på input
   - Använd asynkrona mönster genomgående
   - Stöd samtidiga arbetsflödesinstanser

### 📚 Relaterade koncept:

- **Agent Middleware** - Intercepta agentanrop (tidigare notebook)
- **Hantering av arbetsflödesstatus** - Persistera arbetsflödesstatus mellan körningar
- **Samarbete mellan flera agenter** - Kombinera människa-i-loopen med agentteam
- **Händelsedrivna arkitekturer** - Bygg reaktiva system med händelser

---

### 🎓 Grattis!

Du har bemästrat arbetsflöden med människa-i-loopen i Microsoft Agent Framework! Du vet nu hur man:
- ✅ Pausar arbetsflöden för att samla mänsklig input
- ✅ Använder RequestInfoExecutor och RequestInfoMessage
- ✅ Hanterar strömmande exekvering med händelser
- ✅ Skapar anpassade exekverare med @handler
- ✅ Styr arbetsflöden baserat på mänskliga beslut
- ✅ Bygger interaktiva AI-agenter som samarbetar med människor

**Detta är ett kritiskt mönster för att bygga pålitliga, kontrollerbara AI-system!** 🚀



---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör det noteras att automatiska översättningar kan innehålla fel eller felaktigheter. Det ursprungliga dokumentet på dess ursprungliga språk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
